In [1]:
import quantfreedom as qf
from quantfreedom import _typing as tp
import polars as pl
import pandas as pd
import pyarrow
import numpy as np
import talib


In [2]:
data = pl.read_csv('30min.csv', parse_dates=True)
print(data)

shape: (6960, 6)
┌─────────────────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│ time                ┆ open    ┆ high    ┆ low     ┆ close   ┆ volume  │
│ ---                 ┆ ---     ┆ ---     ┆ ---     ┆ ---     ┆ ---     │
│ datetime[μs]        ┆ f64     ┆ f64     ┆ f64     ┆ f64     ┆ f64     │
╞═════════════════════╪═════════╪═════════╪═════════╪═════════╪═════════╡
│ 2020-10-01 00:00:00 ┆ 10759.0 ┆ 10817.5 ┆ 10759.0 ┆ 10790.0 ┆ 226.42  │
│ 2020-10-01 00:30:00 ┆ 10790.0 ┆ 10790.5 ┆ 10775.0 ┆ 10777.0 ┆ 41.636  │
│ 2020-10-01 01:00:00 ┆ 10777.0 ┆ 10793.0 ┆ 10777.0 ┆ 10793.0 ┆ 23.405  │
│ 2020-10-01 01:30:00 ┆ 10793.0 ┆ 10836.0 ┆ 10792.5 ┆ 10822.0 ┆ 114.161 │
│ ...                 ┆ ...     ┆ ...     ┆ ...     ┆ ...     ┆ ...     │
│ 2021-02-22 22:00:00 ┆ 54818.0 ┆ 54938.0 ┆ 54301.5 ┆ 54314.0 ┆ 584.14  │
│ 2021-02-22 22:30:00 ┆ 54314.0 ┆ 54314.0 ┆ 53773.5 ┆ 54077.0 ┆ 398.019 │
│ 2021-02-22 23:00:00 ┆ 54077.0 ┆ 54208.5 ┆ 53266.0 ┆ 53549.0 ┆ 528.575 │
│ 2021-02-22 23:30:00

In [5]:
s1 = pl.Series("a", [1, 2, 3])
s1

a
i64
1
2
3


In [20]:
type(data['close'])

polars.internals.series.series.Series

In [36]:
sl = data['close']
sl.to_pandas

<bound method Series.to_pandas of shape: (6960,)
Series: 'close' [f64]
[
	10790.0
	10777.0
	10793.0
	10822.0
	10808.5
	10808.5
	10798.5
	10794.5
	10811.0
	10798.5
	10798.0
	10810.5
	...
	52154.0
	52853.0
	53205.5
	53378.5
	53560.0
	54102.5
	53767.5
	54370.5
	54818.0
	54314.0
	54077.0
	53549.0
	54041.0
]>

In [7]:
data['close'].to_numpy()

array([10790., 10777., 10793., ..., 54077., 53549., 54041.])

In [24]:
timeperiods=np.arange(15, 31, 5).tolist()
below_ranges=np.arange(20, 31, 5).tolist()
prices=data['close']
temp_rsi = np.empty((prices.shape[0], len(timeperiods)))
temp_rsi_below = np.empty(
    (prices.shape[0], (len(timeperiods)*len(below_ranges))), dtype=np.bool_)
c = 0

for i in range(len(timeperiods)):
    temp_rsi[:, i] = talib.RSI(
        prices,
        timeperiod=timeperiods[i],
    )
    for x in range(len(below_ranges)):
        temp_rsi_below[:, c] = np.where(
            temp_rsi[:, i] < below_ranges[x], True, False
        )
        c += 1

In [25]:
temp_rsi_below

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [26]:
pl.DataFrame(
    temp_rsi_below,
    # index=prices.index,
    # columns=pl.MultiIndex.from_product(
    #     [below_ranges, timeperiods],
    #     names=['rsi_below', 'rsi_timeperiod'])
)


column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11
bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
false,false,false,false,false,false,false,false,false,false,false,false
false,false,false,false,false,false,false,false,false,false,false,false
false,false,false,false,false,false,false,false,false,false,false,false
false,false,false,false,false,false,false,false,false,false,false,false
false,false,false,false,false,false,false,false,false,false,false,false
false,false,false,false,false,false,false,false,false,false,false,false
false,false,false,false,false,false,false,false,false,false,false,false
false,false,false,false,false,false,false,false,false,false,false,false
false,false,false,false,false,false,false,false,false,false,false,false


In [29]:
pd.DataFrame(
    temp_rsi_below,
    # index=prices.index,
    # columns=pl.MultiIndex.from_product(
    #     [below_ranges, timeperiods],
    #     names=['rsi_below', 'rsi_timeperiod'])
)


,0,1,2,3,4,5,6,7,8,9,10,11
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
6955,False,False,False,False,False,False,False,False,False,False,False,False
6956,False,False,False,False,False,False,False,False,False,False,False,False
6957,False,False,False,False,False,False,False,False,False,False,False,False
6958,False,False,False,False,False,False,False,False,False,False,False,False
